# Generating Initial User-Item Matrix from Pre-Existing GoodReads Ratings

### Data Sources:

`goodbooks-10k`: https://github.com/zygmuntz/goodbooks-10k
- We took a sample of 3625 books (`books.csv`) for our BetterReads database, and all available user ratings (`ratings.csv`) for those books.

`goodreads_descriptions_genres.csv`: https://www.kaggle.com/datasets/ishikajohari/best-books-10k-multi-genre-data
- We obtained genres and book descriptions for our sample of 3625 books.

### The initial user-item matrix (generated at the end of this notebook) contains 53423 users and 3625 books from GoodReads.

In [1]:
import pandas as pd
import pyarrow

In [2]:
gb_ratings_df = pd.read_csv("data/goodbooks-10k/ratings.csv")
gb_ratings_df.head()

# gb_ratings_df[gb_ratings_df["book_id"] == 5397]

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [3]:
gb_books_df = pd.read_csv("data/goodbooks-10k/books.csv")
gb_books_df
# gb_books_df[gb_books_df["work_id"] == 5397]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [4]:
# Merge books and ratings
merged_books_ratings = pd.merge(gb_books_df, gb_ratings_df, left_on='book_id', right_on='book_id', how='inner')
merged_books_ratings.dropna()

merged_books_ratings

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,user_id,rating
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2886,5
1,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,6158,5
2,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,3991,4
3,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,5281,5
4,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,5721,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5976474,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,35336,4
5976475,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,17999,3
5976476,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,49007,4
5976477,10000,8914,8914,11817,31,375700455,9.780376e+12,John Keegan,1998.0,The First World War,...,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...,43319,5


In [5]:
merged_books_ratings_cleaned = merged_books_ratings[["isbn", "user_id", "rating"]]
merged_books_ratings_cleaned.head()

,isbn,user_id,rating
0,439023483,2886,5
1,439023483,6158,5
2,439023483,3991,4
3,439023483,5281,5
4,439023483,5721,5


In [6]:
# read in better_reads_data_processed.json (processed in BetterReads_Initial_User_Item_Matrix.ipynb)
better_reads_df = pd.read_json("data/better_reads_data_processed.json")
better_reads_df

,title,author,publishYear,image,description,genre,ISBN,numberOfEditions,averageRating,ratingsCount,reviewCount
0,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,2008,https://images.gr-assets.com/books/1447303603m...,"Could you survive on your own in the wild, wit...","[Young Adult, Fiction, Dystopia, Fantasy, Scie...",439023483,272,4.34,4780653,155254
1,To Kill a Mockingbird,Harper Lee,1960,https://images.gr-assets.com/books/1361975680m...,The unforgettable novel of a childhood in a sl...,"[Classics, Fiction, Historical Fiction, School...",61120081,487,4.25,3198671,72586
2,The Great Gatsby,F. Scott Fitzgerald,1925,https://images.gr-assets.com/books/1490528560m...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"[Classics, Fiction, School, Historical Fiction...",743273567,1356,3.89,2683664,51992
3,The Fault in Our Stars,John Green,2012,https://images.gr-assets.com/books/1360206420m...,Despite the tumor-shrinking medical miracle th...,"[Young Adult, Romance, Fiction, Contemporary, ...",525478817,226,4.26,2346404,140739
4,The Hobbit,J.R.R. Tolkien,1937,https://images.gr-assets.com/books/1372847500m...,"First published over 50 years ago, J.R.R. Tolk...","[Fantasy, Graphic Novels, Classics, Fiction, C...",618260307,969,4.25,2071616,37653
...,...,...,...,...,...,...,...,...,...,...,...
3620,A Theory of Justice,John Rawls,1971,https://s.gr-assets.com/assets/nophoto/book/11...,"Since it appeared in 1971, John Rawls's A Theo...","[Philosophy, Politics, Nonfiction, Law, Classi...",674017722,44,3.91,8472,168
3621,"The Red Knight (The Traitor Son Cycle, #1)",Miles Cameron,2012,https://images.gr-assets.com/books/1348037761m...,Twenty eight florins a month is a huge price t...,"[Fantasy, Epic Fantasy, Fiction, High Fantasy,...",575113294,26,4.12,8556,742
3622,The Time Traveller's Guide to Medieval England...,Ian Mortimer,2008,https://images.gr-assets.com/books/1328167619m...,Imagine you could get into a time machine and ...,"[History, Nonfiction, Medieval, Historical, Br...",224079948,27,3.99,9824,970
3623,"The Everafter War (The Sisters Grimm, #7)","Michael Buckley, Peter Ferguson",2009,https://images.gr-assets.com/books/1388278230m...,The Grimm family is caught in the midst of a w...,"[Fantasy, Middle Grade, Fairy Tales, Mystery, ...",810983559,2,4.32,12493,455


In [7]:
# keep just the ratings for the 3625 books in the betterreads database
reduced_merged_books_ratings = pd.merge(better_reads_df, merged_books_ratings_cleaned, left_on='ISBN', right_on='isbn', how='inner')
reduced_merged_books_ratings_cleaned = reduced_merged_books_ratings[["isbn", "user_id", "rating"]]
reduced_merged_books_ratings_cleaned

,isbn,user_id,rating
0,439023483,2886,5
1,439023483,6158,5
2,439023483,3991,4
3,439023483,5281,5
4,439023483,5721,5
...,...,...,...
3809350,61711527,22360,4
3809351,61711527,13990,3
3809352,61711527,24615,5
3809353,61711527,47930,4


In [8]:
# read in ISBN to book_id mappings generated from script (better-reads/backend/scripts/getBookIdToIsbnMap.js) run on MongoDB database
isbn_book_id = pd.read_json("data/bookIdToIsbnMap.json")
isbn_book_id

,6861eac7f73dac107c42d926,6861eac7f73dac107c42d927,6861eac7f73dac107c42d928,6861eac7f73dac107c42d929,6861eac7f73dac107c42d92a,6861eac7f73dac107c42d92b,6861eac7f73dac107c42d92c,6861eac7f73dac107c42d92d,6861eac7f73dac107c42d92e,6861eac7f73dac107c42d92f,...,6861eac7f73dac107c42e745,6861eac7f73dac107c42e746,6861eac7f73dac107c42e747,6861eac7f73dac107c42e748,6861eac7f73dac107c42e749,6861eac7f73dac107c42e74a,6861eac7f73dac107c42e74b,6861eac7f73dac107c42e74c,6861eac7f73dac107c42e74d,6861eac7f73dac107c42e74e
ISBN,439023483,61120081,743273567,525478817,618260307,316769177,679783261,1594480001,62024035,451524934,...,739416383,679749845,1580050751,679750150,140255087,674017722,575113294,224079948,810983559,61711527
title,"The Hunger Games (The Hunger Games, #1)",To Kill a Mockingbird,The Great Gatsby,The Fault in Our Stars,The Hobbit,The Catcher in the Rye,Pride and Prejudice,The Kite Runner,"Divergent (Divergent, #1)",1984,...,"The Prize Winner of Defiance, Ohio: How My Mot...",The Comfort of Strangers,Cunt: A Declaration of Independence,The Sailor Who Fell from Grace with the Sea,The Snow Leopard,A Theory of Justice,"The Red Knight (The Traitor Son Cycle, #1)",The Time Traveller's Guide to Medieval England...,"The Everafter War (The Sisters Grimm, #7)",Cinderella Ate My Daughter: Dispatches from th...
author,Suzanne Collins,Harper Lee,F. Scott Fitzgerald,John Green,J.R.R. Tolkien,J.D. Salinger,Jane Austen,Khaled Hosseini,Veronica Roth,"George Orwell, Erich Fromm, Celâl Üster",...,Terry Ryan,Ian McEwan,"Inga Muscio, Betty Dodson","Yukio Mishima, John Nathan",Peter Matthiessen,John Rawls,Miles Cameron,Ian Mortimer,"Michael Buckley, Peter Ferguson",Peggy Orenstein


In [9]:
# transpose mapping so ISBN and book_id are in their own columns
isbn_book_id_trans = isbn_book_id.T
isbn_book_id_map = isbn_book_id_trans.reset_index()
isbn_book_id_map

,index,ISBN,title,author
0,6861eac7f73dac107c42d926,439023483,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins
1,6861eac7f73dac107c42d927,61120081,To Kill a Mockingbird,Harper Lee
2,6861eac7f73dac107c42d928,743273567,The Great Gatsby,F. Scott Fitzgerald
3,6861eac7f73dac107c42d929,525478817,The Fault in Our Stars,John Green
4,6861eac7f73dac107c42d92a,618260307,The Hobbit,J.R.R. Tolkien
...,...,...,...,...
3620,6861eac7f73dac107c42e74a,674017722,A Theory of Justice,John Rawls
3621,6861eac7f73dac107c42e74b,575113294,"The Red Knight (The Traitor Son Cycle, #1)",Miles Cameron
3622,6861eac7f73dac107c42e74c,224079948,The Time Traveller's Guide to Medieval England...,Ian Mortimer
3623,6861eac7f73dac107c42e74d,810983559,"The Everafter War (The Sisters Grimm, #7)","Michael Buckley, Peter Ferguson"


In [10]:
# replace ISBN with MongoDB book_id
reduced_merged_books_ratings_book_id = pd.merge(reduced_merged_books_ratings_cleaned, isbn_book_id_map, left_on='isbn', right_on='ISBN', how='inner')
reduced_merged_books_ratings_book_id
ratings_books = reduced_merged_books_ratings_book_id[["index", "user_id", "rating"]]
ratings_books = ratings_books.rename(columns={'index': 'book_id'})
ratings_books

,book_id,user_id,rating
0,6861eac7f73dac107c42d926,2886,5
1,6861eac7f73dac107c42d926,6158,5
2,6861eac7f73dac107c42d926,3991,4
3,6861eac7f73dac107c42d926,5281,5
4,6861eac7f73dac107c42d926,5721,5
...,...,...,...
3809350,6861eac7f73dac107c42e74e,22360,4
3809351,6861eac7f73dac107c42e74e,13990,3
3809352,6861eac7f73dac107c42e74e,24615,5
3809353,6861eac7f73dac107c42e74e,47930,4


In [11]:
# Pivot mapping dataframe to user-item matrix
matrix = ratings_books.pivot_table(index="user_id", columns="book_id", values="rating").fillna(0)
matrix.shape

(53423, 3625)

In [12]:
# export user-item matrix created from goodreads dataset
# matrix.to_json("data/initial-user-item-matrix.json")

In [13]:
# export user-item matrix created from goodreads dataset to parquet (more space- and time- efficient than json)
matrix.to_parquet("data/initial-user-item-matrix.parquet", engine="pyarrow")